# Importing Necessary Libraries


In [2]:
import pandas as pd
from prophet import Prophet



Importing plotly failed. Interactive plots will not work.


* Loading relevant Data into DataFrames

In [3]:
df_cpu=pd.read_csv('DatasetsForDemandForecasting/CPU_train.csv')
df_gpu=pd.read_csv('DatasetsForDemandForecasting/GPU_train.csv')
df_mobo=pd.read_csv('DatasetsForDemandForecasting/MOBO_train.csv')
df_ram=pd.read_csv('DatasetsForDemandForecasting/RAM_train.csv')  
df_psu=pd.read_csv('DatasetsForDemandForecasting/PSU_train.csv')
df_storage=pd.read_csv('DatasetsForDemandForecasting/Storage_train.csv')
df_case=pd.read_csv('DatasetsForDemandForecasting/Case_train.csv')    
'''7 types of Parts'''

'7 types of Parts'

# In the cell Below we 
1. Load a dataFrame with the dates for which we want to predict demand
2. Also load a holidays DataFrame to handle seasonality 
3. Define function to create model and store result of predictions 
4. Call the defined function 

In [6]:


# DataFrame containing future dates for prediction
df_future_dates = pd.read_csv('DatasetsForDemandForecasting/test_dates.csv')
predictions_by_product = {}
holidays_df = pd.read_csv('holidays.csv')
def fit_and_predict_by_product(df, future_dates=df_future_dates,holidays_funct=holidays_df):
    predictions_by_product = {}
    models_by_product = {}

    for product_id in df['Prod_id'].unique():
        df_product = df[df['Prod_id'] == product_id]

        # Create and fit a new Prophet model for each product
        prophet_model = Prophet(holidays=holidays_funct)
        prophet_model.fit(df_product[['ds', 'y']])
        models_by_product[product_id] = prophet_model
        # Make predictions for future dates
        future_df = future_dates
        predictions = prophet_model.predict(future_df)

        # Store the predictions in the dictionary
        predictions_by_product[product_id] = predictions[['ds', 'yhat']]

    return predictions_by_product,models_by_product

cpu_pred,cpu_model=fit_and_predict_by_product(df_cpu)
case_pred,case_model=fit_and_predict_by_product(df_case)
ram_pred,ram_model=fit_and_predict_by_product(df_ram)
storage_pred,storage_model=fit_and_predict_by_product(df_storage)
gpu_pred,gpu_model=fit_and_predict_by_product(df_gpu)
psu_pred,psu_model=fit_and_predict_by_product(df_psu)
mobo_pred,mobo_model=fit_and_predict_by_product(df_mobo)





18:20:27 - cmdstanpy - INFO - Chain [1] start processing
18:20:27 - cmdstanpy - INFO - Chain [1] done processing
18:20:27 - cmdstanpy - INFO - Chain [1] start processing
18:20:27 - cmdstanpy - INFO - Chain [1] done processing
18:20:27 - cmdstanpy - INFO - Chain [1] start processing
18:20:28 - cmdstanpy - INFO - Chain [1] done processing
18:20:28 - cmdstanpy - INFO - Chain [1] start processing
18:20:28 - cmdstanpy - INFO - Chain [1] done processing
18:20:28 - cmdstanpy - INFO - Chain [1] start processing
18:20:29 - cmdstanpy - INFO - Chain [1] done processing
18:20:29 - cmdstanpy - INFO - Chain [1] start processing
18:20:29 - cmdstanpy - INFO - Chain [1] done processing
18:20:29 - cmdstanpy - INFO - Chain [1] start processing
18:20:30 - cmdstanpy - INFO - Chain [1] done processing
18:20:30 - cmdstanpy - INFO - Chain [1] start processing
18:20:30 - cmdstanpy - INFO - Chain [1] done processing
18:20:30 - cmdstanpy - INFO - Chain [1] start processing
18:20:30 - cmdstanpy - INFO - Chain [1]

* As we have finished with model training and have stored the results of our prediction we now extract the results to 

In [29]:
def extract_to_csv(temp_dict,filename):
    output_directory = 'PredictionsBYProdId/'
    i=0
    result_dict={}
    for key, dataframe in temp_dict.items():
       
        dataframe['Prod_id']=key
        result_dict[key]=dataframe
    
    dfs_to_concat = [dataframe for dataframe in result_dict.values()]
    final_df = pd.concat(dfs_to_concat, ignore_index=True)
    final_df = final_df.set_index('Prod_id').reset_index()
    print('All Loops completed')
    final_df.to_csv('{}.csv'.format(filename),index=False)

extract_to_csv(cpu_pred,'CPU_pred')
extract_to_csv(case_pred,'Case_pred')
extract_to_csv(ram_pred,'RAM_pred')
extract_to_csv(storage_pred,'Storage_pred')
extract_to_csv(gpu_pred,"GPU_Pred")
extract_to_csv(psu_pred,'PSU_Pred')
extract_to_csv(mobo_pred,'MOBO_pred')

All Loops completed
All Loops completed
All Loops completed
All Loops completed
All Loops completed
All Loops completed
All Loops completed


In [17]:
from datetime import datetime
df_cpu_test=pd.read_csv('DatasetsForDemandForecasting/CPU_test.csv')
df_case_test=pd.read_csv('DatasetsForDemandForecasting/Case_test.csv')
df_gpu_test=pd.read_csv('DatasetsForDemandForecasting/GPU_test.csv')
df_mobo_test=pd.read_csv('DatasetsForDemandForecasting/MOBO_test.csv')
df_ram_test=pd.read_csv('DatasetsForDemandForecasting/RAM_test.csv')
df_storage_test=pd.read_csv('DatasetsForDemandForecasting/Storage_test.csv')
df_psu_test=pd.read_csv('DatasetsForDemandForecasting/PSU_test.csv')

def calculate_rmse(df_test, predictions_dict):
    rmse_arr = []
    mse_arr = []
    mae_arr = []

    for key, df_preds in predictions_dict.items():
        # Access the DataFrame for the current key
        df_test_product = df_test[df_test['Prod_id'] == key]

        # Filter predictions for dates up to December 2022
        df_preds = df_preds[df_preds['ds'] <= datetime(2022, 12, 1)]

        # Calculate absolute error
        abs_error = [abs(y_true - y_pred) for y_true, y_pred in zip(df_test_product['y'], df_preds['yhat'])]

        # Calculate squared error for MSE
        squared_error = [error ** 2 for error in abs_error]

        # Append errors to arrays
        mae_arr.append(sum(abs_error) / len(abs_error))
        mse_arr.append(sum(squared_error) / len(squared_error))
        rmse_arr.append((sum(squared_error) / len(squared_error)) ** 0.5)

    # Calculate overall metrics
    rmse = sum(rmse_arr) / len(rmse_arr)
    mse = sum(mse_arr) / len(mse_arr)
    mae = sum(mae_arr) / len(mae_arr)

    return rmse, mse, mae

rmse_cpu,mse_cpu,mae_cpu=calculate_rmse(df_cpu_test,cpu_pred)
rmse_case,mse_case,mae_case=calculate_rmse(df_case_test,case_pred)
rmse_gpu,mse_gpu,mae_gpu=calculate_rmse(df_gpu_test,gpu_pred)
rmse_mobo,mse_mobo,mae_mobo=calculate_rmse(df_mobo_test,mobo_pred)
rmse_ram,mse_ram,mae_ram=calculate_rmse(df_ram_test,ram_pred)
rmse_storage,mse_storage,mae_storage=calculate_rmse(df_storage_test,storage_pred)
rmse_psu,mse_psu,mae_psu=calculate_rmse(df_psu_test,psu_pred)
                            
        


In [21]:

print('For CPU RMSE:: {} MSE:: {} MAE:: {}'.format(rmse_cpu, mse_cpu, mae_cpu))
print('For Case RMSE:: {} MSE:: {} MAE:: {}'.format(rmse_case, mse_case, mae_case))
print('For GPU RMSE:: {} MSE:: {} MAE:: {}'.format(rmse_gpu, mse_gpu, mae_gpu))
print('For Motherboard RMSE:: {} MSE:: {} MAE:: {}'.format(rmse_mobo, mse_mobo, mae_mobo))
print('For RAM RMSE:: {} MSE:: {} MAE:: {}'.format(rmse_ram, mse_ram, mae_ram))
print('For Storage RMSE:: {} MSE:: {} MAE:: {}'.format(rmse_storage, mse_storage, mae_storage))
print('For PSU RMSE:: {} MSE:: {} MAE:: {}'.format(rmse_psu, mse_psu, mae_psu))


For CPU RMSE:: 29.23824725524627 MSE:: 864.2007341387681 MAE:: 24.58025512377865
For Case RMSE:: 29.46106524602948 MSE:: 876.939124962187 MAE:: 24.819658901634085
For GPU RMSE:: 33.01185918056074 MSE:: 1101.7133961944348 MAE:: 27.262772639585076
For Motherboard RMSE:: 29.28139067748505 MSE:: 865.9539160324892 MAE:: 24.647924613521347
For RAM RMSE:: 29.34384547243508 MSE:: 869.7716399698896 MAE:: 24.71789195901869
For Storage RMSE:: 29.455046837977992 MSE:: 876.3308757981602 MAE:: 24.81635773123925
For PSU RMSE:: 29.329433802953492 MSE:: 869.1001141492097 MAE:: 24.666094030884338
